In [264]:
from core.general import *

In [265]:
filename = 'Education'

In [266]:
template = '\n'.join([i.strip() for i in read_from_file(f'{filename}.md').replace('---', '').replace('- ', '<br>- ').split('\n')])
template = template.replace('\r', '')
while '\n\n\n' in template:
    template = template.replace('\n\n\n', '\n\n')

template = template.replace('\n\n', '\n<br><br>\n')

info(template)

INFO  # Overview
<br><br>
Through our surveys and questionnaires, we uncovered a significant gap in the public’s understanding of biofilms, biotechnology, and synthetic biology. This insight led us to make it a priority to **break down the barriers that prevent people from accessing cutting-edge knowledge in these fields**. We realized that many individuals, from high school students to adults, lacked even basic awareness about biofilms and their impact on healthcare, and this knowledge gap needed to be addressed.
<br><br>
To close this gap, we organized a **comprehensive educational workshop aimed at high school students**. This workshop was designed to provide students with an in-depth understanding of biofilms, synthetic biology, and their real-world applications, ensuring that the content was accessible and engaging. At the conclusion of the workshop, we **gathered valuable feedback** from the participants to refine and improve our educational approach. We also sought additional in

In [267]:
def fold_if(x, check, concat = False) -> list:
    ans = []
    for i in x:
        if check(i):
            ans.append([i, []])
        else:
            if len(ans) == 0:
                continue
            ans[-1][1].append(i)
    if concat:
        ans = [[i[0], '\n'.join(i[1])] for i in ans]
    return ans

In [268]:
def md2html(x: str) -> str:
    tmp = x.split('**')
    ans = tmp[0]
    o = False
    for i in tmp[1:]:
        if not o:
            ans = f'{ans}<b>{i}'
        else:
            ans = f'{ans}</b>{i}'
        o = not o
    return ans

def md3html(x: str) -> str:
    tmp = x.split('*')
    ans = tmp[0]
    o = False
    for i in tmp[1:]:
        if not o:
            ans = f'{ans}<i>{i}'
        else:
            ans = f'{ans}</i>{i}'
        o = not o
    return ans

template = md3html(md2html(template))

In [269]:
def fold_two(x):
    fold_if(x, lambda x: x.startswith('## '))

In [270]:
content_id = 0
def parse_content(x: str):
    global content_id
    last = []
    for i in x.split('\n'):
        if len(last) == 0:
            last = [[i]]
            continue
        if i.startswith('[PDF]'):
            last.append([i])
            last.append([])
            continue
        if i.startswith('!['):
            last.append([i])
            last.append([])
            continue
        if i.startswith('- '):
            last.append([' - '])
            last.append([i[2:]])
            continue
        if i.strip().startswith('- '):
            last.append([' -- '])
            last.append([i[2:]])
            continue
        last[-1].append(i)
    while len(last) != -1 and len(last[-1]) == 0:
        last.pop()
    
    # for i in last:
    #     if not len(i) == 0 and i[0].strip() == '<br>': i = i[1:];
    #     if not len(i) == 0 and i[-1].strip() == '<br>': i.pop();

    last = ['\n'.join(i).strip().replace('\n\n', '\n<br><br>\n') for i in last]
    ans = []
    for i in last:
        if i.startswith('!['):
            content_id = content_id + 1
            ans.append({'type': 'img', 'text': content_id})
        elif i.startswith('[PDF]'):
            debug({'type': 'pdf', 'text': i[6:-1]})
            ans.append({'type': 'pdf', 'text': i[6:-1]})
        elif i == '-':
            ans.append({'type': 'icon', 'text': 'fa-solid fa-check'})
        elif i == '--':
            ans.append({'type': 'subicon', 'text': 'fa-solid fa-check'})
        else:
            ans.append({'type': 'text', 'text': i})
    return ans

In [271]:
def foldtt(x, t: int):
    token = '#' * t + ' '
    if token not in x:
        return [parse_content(x), None]
    tmp = (fold_if(x.split('\n'), lambda x: x.startswith(token), concat = True))
    des = parse_content(x[:x.find(token)])
    tmp = [[j[0][t + 1:], j[1]] for j in tmp]
    ans = []
    for i in tmp:
        now = foldtt(i[1], t + 1)
        l = {
            'level': t,
            'title': i[0][:],
            'description': now[0],
            'text': now[1]
        }
        ans.append(l)
        debug(l)
    return [des, ans]
    
f = foldtt(template, 1)

f

DEBUG {'level': 3, 'title': 'The Rules of the game', 'description': [{'type': 'text', 'text': '<br><br>\nWe developed an educational tower defense game where the enemies are bacteria equipped with one to three layers of biofilm shields.\nThe bacterial species include <i>Staphylococcus aureus</i>, <i>Mycobacterium</i>, <i>Escherichia coli</i>, <i>Pseudomonas aeruginosa</i>, and <i>Streptococcus</i>.\n<br><br>\nThere are two types of defense towers: enzyme towers and antibiotic towers.\n<br><br>\nEnzyme towers can only target biofilm shields:\n<br><br>\nDispersin B Tower attacks PNAG-based polysaccharide shields.\n<br><br>\nProteinase K Tower targets protein shields.\n<br><br>\nDNase I Tower breaks down eDNA shields.\n<br><br>\nOnly the outermost shield can be attacked at any given time.\n<br><br>\nOnce a bacterium’s shields are completely destroyed, enzyme towers can no longer deal damage.\nAt this stage, players must deploy antibiotic towers to eliminate the unshielded bacteria.\nHowev

[[{'type': 'text', 'text': ''}],
 [{'level': 1,
   'title': 'Overview',
   'description': [{'type': 'text',
     'text': '<br><br>\nThrough our surveys and questionnaires, we uncovered a significant gap in the public’s understanding of biofilms, biotechnology, and synthetic biology. This insight led us to make it a priority to <b>break down the barriers that prevent people from accessing cutting-edge knowledge in these fields</b>. We realized that many individuals, from high school students to adults, lacked even basic awareness about biofilms and their impact on healthcare, and this knowledge gap needed to be addressed.\n<br><br>\nTo close this gap, we organized a <b>comprehensive educational workshop aimed at high school students</b>. This workshop was designed to provide students with an in-depth understanding of biofilms, synthetic biology, and their real-world applications, ensuring that the content was accessible and engaging. At the conclusion of the workshop, we <b>gathered val

In [272]:
for i in [i for i in ls() if i.endswith('.md')]:
    global content_id
    content_id = 0
    filename = i.replace('.md', '')
    template = '\n'.join([i.strip() for i in read_from_file(f'{filename}.md').replace('---', '').replace('- ', '\n\n- ').replace('<aside>', '').replace('</aside>', '').split('\n')])
    template = template.replace('\r', '')
    while '\n\n\n' in template:
        template = template.replace('\n\n\n', '\n\n')

    # template = template.replace('\n\n', '\n<br><br>\n')

    # info(template)
    template = md3html(md2html(template))

    f = foldtt(template, 1)
    now = read_from_file('../../src/hp/template.vue')

    write_to_file(f'../../src/hp/{filename.lower().replace(' ', '-')}/App.vue', now.replace('<name>', filename).replace('<data>', str(f).replace('None', 'undefined')))

DEBUG {'level': 2, 'title': 'NYCU-Formosa', 'description': [{'type': 'text', 'text': '> We held an online meeting with the NYCU iGEM team to share project progress and discuss possible collaboration ideas. Their project focused on optimizing <i>plasmid integration</i> techniques for genome editing, combining biology and electrical engineering expertise.\n>'}, {'type': 'img', 'text': 1}, {'type': 'text', 'text': ''}, {'type': 'icon', 'text': 'fa-solid fa-check'}, {'type': 'text', 'text': '<b>Reflection</b>\n<br><br>\nWe realized that our presentation could be improved to convey complex ideas more clearly and engage our peers better.'}, {'type': 'icon', 'text': 'fa-solid fa-check'}, {'type': 'text', 'text': '<b>Learning Points</b>'}, {'type': 'icon', 'text': 'fa-solid fa-check'}, {'type': 'text', 'text': 'Learned about NYCU’s innovative approach to genome integration and their efficient team structure.'}, {'type': 'icon', 'text': 'fa-solid fa-check'}, {'type': 'text', 'text': 'Inspired b

In [273]:
import pyperclip as pp

pp.copy(str(f).replace('None', 'undefined'))

In [274]:
now = read_from_file('../../src/hp/template.vue')

write_to_file(f'../../src/hp/{filename.lower().replace(' ', '-')}/App.vue', now.replace('<name>', filename).replace('<data>', str(f).replace('None', 'undefined')))